In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from keras.datasets import mnist
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

from keras.utils import to_categorical
from keras import layers
from keras import models
from keras import optimizers

Using TensorFlow backend.


In [41]:
import Image_Preprocess
BCC_train=Image_Preprocess.BCC_train()
BCC_test=Image_Preprocess.BCC_test()
FCC_train=Image_Preprocess.FCC_train()
FCC_test=Image_Preprocess.FCC_test()

In [42]:
FCC_test=np.array(FCC_test)
BCC_test=np.array(BCC_test)
FCC_train=np.array(FCC_train)
BCC_train=np.array(BCC_train)


In [43]:
# making training data and test data

train_data=np.append(FCC_train,BCC_train,axis=0)
test_data=np.append(FCC_test,BCC_test,axis=0)

# making train_label
train_label=[]
for i in range(19):
    train_label.append('FCC')
for i in range(19):
    train_label.append('BCC')
    
test_label=[]
for i in range(2):
    test_label.append('FCC')
for i in range(2):
    test_label.append('BCC')
    
train_label=np.array(train_label)
test_label=np.array(test_label)


In [44]:
# reshaping data
train_data = train_data.reshape(-1,400,300,1).astype('float32')/255
test_data = test_data.reshape(-1,400,300,1).astype('float32')/255

In [45]:
# one hot encoder
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train_label_10 = label_encoder.fit_transform(train_label)
test_label_10 = label_encoder.fit_transform(test_label)
train_label = to_categorical(train_label_10)
test_label = to_categorical(test_label_10)

In [46]:
# cross validation
train_data,val_data,train_label,val_label=train_test_split(train_data,train_label,test_size=0.25,random_state=13)

In [48]:
train_data.shape

(28, 400, 300, 1)

In [49]:
train_label.shape

(28, 2)

In [ ]:
# fitting model, 測試1
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(train_data)

# fits the model on batches with real-time data augmentation:
model.fit_generator(datagen.flow(train_data, train_label, batch_size=32),
                    steps_per_epoch=len(train_data) / 32, epochs=12)

Epoch 1/12


In [50]:
from keras.models import Sequential
from keras.layers import Convolution2D,MaxPool2D,Flatten,Dense,Dropout
from keras.callbacks import TensorBoard

model=Sequential([
    Convolution2D(32,3,3,input_shape=(400,300,1),activation='relu'),
    MaxPool2D(pool_size=(2,2)),
    Convolution2D(64,3,3,input_shape=(400,300,1),activation='relu'),
    MaxPool2D(pool_size=(2,2)),
    Flatten(),
    Dense(64,activation='relu'),
    Dropout(0.5),
    Dense(2,activation='sigmoid')
])

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.summary()




_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 398, 298, 32)      320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 199, 149, 32)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 197, 147, 64)      18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 98, 73, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 457856)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                29302848  
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
__________

/Users/hsiaoyuhsien/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(400, 300,..., activation="relu")`
  
/Users/hsiaoyuhsien/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), input_shape=(400, 300,..., activation="relu")`
  


In [28]:
from keras.preprocessing.image import ImageDataGenerator
      
train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
#test_datagen = ImageDataGenerator(rescale = 1./255)
training_set = train_datagen.flow_from_directory(FCC_test, target_size = (128, 128), batch_size = 32, class_mode = 'binary')
#test_set = test_datagen.flow_from_directory('dataset/test_set', target_size = (128, 128), batch_size = 32, class_mode = 'binary')
#model.fit_generator(training_set, samples_per_epoch = 400, nb_epoch = 30, validation_data = test_set, nb_val_samples = 100)

ValueError: listdir: embedded null character in path